# タイムスタンプの処理

APIのタイムスタンプは、UNIX（エポック秒）時間です。UNIX時間は協定世界時 (UTC) での1970年1月1日午前0時0分0秒からの経過秒数を指します。

ここでは、タイムスタンプを協定世界時に変換します。

`get_transactions` メソッドから、2017年3月13日のbtc_jpyを取得します。

In [1]:
import python_bitbankcc

public = python_bitbankcc.public()
transactions = public.get_transactions("btc_jpy", "20170313")
transactions["transactions"][:5]

[{'transaction_id': 66846,
  'side': 'buy',
  'price': '142476',
  'amount': '0.0008',
  'executed_at': 1489363215164},
 {'transaction_id': 66847,
  'side': 'buy',
  'price': '142498',
  'amount': '0.0002',
  'executed_at': 1489363229612},
 {'transaction_id': 66848,
  'side': 'buy',
  'price': '142496',
  'amount': '0.0003',
  'executed_at': 1489363244589},
 {'transaction_id': 66849,
  'side': 'buy',
  'price': '142500',
  'amount': '0.0002',
  'executed_at': 1489363258597},
 {'transaction_id': 66850,
  'side': 'buy',
  'price': '142474',
  'amount': '0.0010',
  'executed_at': 1489363275582}]

ここでは、最初のタイムスタンプを変換します。

In [2]:
timestamp = transactions["transactions"][0]["executed_at"]
timestamp


1489363215164

## タイムスタンプから日付への変換

Python標準ライブラリの `datetime.datetime.fromtimestamp` 関数は、POSIXタイムスタンプに対応するローカルな日付を返します。

```{seealso}
Python公式ドキュメント

datetime --- 基本的な日付と時間の型

https://docs.python.org/ja/3/library/datetime.html#datetime.date.fromtimestamp
```

In [3]:
import datetime

datetime.datetime.fromtimestamp(timestamp * 0.001)

datetime.datetime(2017, 3, 13, 9, 0, 15, 164000)

## pandasのDataFrameから一括変換

ここではpandasのDataFrameに読み込み、executed_at列を日付型に変換します。

In [4]:
import pandas as pd

df = pd.DataFrame(transactions["transactions"])
df.head()

,transaction_id,side,price,amount,executed_at
0,66846,buy,142476,0.0008,1489363215164
1,66847,buy,142498,0.0002,1489363229612
2,66848,buy,142496,0.0003,1489363244589
3,66849,buy,142500,0.0002,1489363258597
4,66850,buy,142474,0.0010,1489363275582


タイムスタンプを日付型に変換する `timestamp_to_datetime` 関数を作成します。

In [5]:
def timestamp_to_datetime(timestamp: int) -> datetime.datetime:
    return datetime.datetime.fromtimestamp(timestamp * 0.001)

`pandas.Series` の `map` メソッドは、引数に渡した関数をSeriesの各要素に対して実行します。

In [6]:
df.loc[:, "executed_at"].map(timestamp_to_datetime)

0      2017-03-13 09:00:15.164
1      2017-03-13 09:00:29.612
2      2017-03-13 09:00:44.589
3      2017-03-13 09:00:58.597
4      2017-03-13 09:01:15.582
                 ...          
7956   2017-03-14 08:58:51.595
7957   2017-03-14 08:59:13.581
7958   2017-03-14 08:59:28.122
7959   2017-03-14 08:59:42.161
7960   2017-03-14 08:59:57.156
Name: executed_at, Length: 7961, dtype: datetime64[ns]

`pandas.DataFrame` の `assign` メソッドは列を追加して返します。

In [7]:
df.assign(datetime=df.loc[:, "executed_at"].map(timestamp_to_datetime))

,transaction_id,side,price,amount,executed_at,datetime
0,66846,buy,142476,0.0008,1489363215164,2017-03-13 09:00:15.164
1,66847,buy,142498,0.0002,1489363229612,2017-03-13 09:00:29.612
2,66848,buy,142496,0.0003,1489363244589,2017-03-13 09:00:44.589
3,66849,buy,142500,0.0002,1489363258597,2017-03-13 09:00:58.597
4,66850,buy,142474,0.0010,1489363275582,2017-03-13 09:01:15.582
...,...,...,...,...,...,...
7956,74802,buy,143850,0.0002,1489449531595,2017-03-14 08:58:51.595
7957,74803,buy,143847,0.0001,1489449553581,2017-03-14 08:59:13.581
7958,74804,buy,143847,0.0002,1489449568122,2017-03-14 08:59:28.122
7959,74805,buy,143847,0.0007,1489449582161,2017-03-14 08:59:42.161
